<a href="https://colab.research.google.com/github/LashawnFofung/RAG-Pipelines/blob/main/src/Task_Build_and_Optimize_a_RAG_Pipeline_for_Document_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📑 Build And Optimize A RAG Pipeline For Document Retrieval**

**Data:** *LenderFeesWorksheetNew.pdf*
<br><br>


This Colab notebook sets up a Retrieval-Augmented Generation (RAG) system designed to answer questions about the uploaded LenderFeesWorksheetNew.pdf.
<br><br>

It works by first using PyMuPDF to accurately extract (parse) the text from the PDF. This text is then broken down into smaller, meaningful chunks (✂️) and converted into numerical vectors (🔢) using the efficient `all-MiniLM-L6-v2` HuggingFace embedding model. These vectors are stored in a Vector Index. When you ask a question, the system searches the index to retrieve (🔍) the most relevant text chunks (Vector Retrieval) and feeds both the question and those chunks to the Gemini 2.5 Flash LLM. The LLM then reads the retrieved context and generates a precise answer, such as calculating the total monthly payment or identifying specific fees.

<br><br>


**Table of Contents**
* [1. RAG Pipeline](#scrollTo=Bfr8FNjnSvW8&line=1&uniqifier=1)
  * [1.1 Installation](#scrollTo=CVLftbBrTtJD&line=1&uniqifier=1)
  * [1.2 Setup Environment and Imports](#scrollTo=A-7Tha4LT_Wp&line=1&uniqifier=1)
  * [1.3 API Key Setup](#scrollTo=p43QyjfpUZ-k&line=1&uniqifier=1)
  * [1.4 Document Upload](#scrollTo=zF41iRd_UmH7&line=1&uniqifier=1)
  * [1.5 Custom PyMuPDF Loader Function (PDF Parsing)](#scrollTo=bh0AOQB7nvmp&line=1&uniqifier=1)
  * [1.6 Configure RAG Pipeline (LLM, Embedding, Chunking)](#scrollTo=YqYMeoCroEGh&line=1&uniqifier=1)
  * [1.7 Indexing and Index Creation](#scrollTo=7F7-so82oU_z&line=1&uniqifier=1)
* [2. Reasons for Methods: Embedding, Chunking Strategy, & Retrieval Method](#scrollTo=2lo9ffUJSzJE&line=1&uniqifier=1)
* [3. Create Query Engine and Execute Prompts](#scrollTo=KZt_12xPS5hx)
  * [Prompt 1: "What is the total estimated monthly payment?"](#scrollTo=Jv-P-qLLVPeG&line=1&uniqifier=1)
  * [Prompt 2: "How much does the borrower pay for lender's title insurance?"](#scrollTo=a2xtjKotVX_d)

# **1. RAG Pipeline**

## 1.1 Installation

In [ ]:
# Install LlamaIndex core, Gemini LLM connector, PyMuPDF, and HuggingFace Embedding integration
!pip install -q llama-index llama-index-llms-gemini pymupdf llama-index-embeddings-huggingface nest_asyncio sentence-transformers

## 1.2 Setup Environment and Imports

In [ ]:
import os
import fitz # PyMuPDF
from google.colab import files, userdata
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from typing import List
import nest_asyncio

# Fix event loop conflicts in Colab
nest_asyncio.apply()


## 1.3 API Key Setup

In [ ]:
# Gemini API Key setup in Colab secret
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    if not API_KEY:
        raise ValueError("GEMINI_API_KEY not found in Colab Secrets. Please set it.")
    # Set the official environment variable name required by the Google GenAI SDK
    os.environ["GOOGLE_API_KEY"] = API_KEY
    print("✅ API Key successfully loaded and set as GOOGLE_API_KEY.")
except Exception as e:
    print(f"⚠️ Warning: Could not load API Key from Colab Secrets: {e}")
    print("Please ensure your API Key is set as a Colab secret named 'GEMINI_API_KEY' or set the environment variable manually.")
    # Fallback/Manual setting (Uncomment and replace if Colab Secrets is not used)
    # os.environ["GOOGLE_API_KEY"] = "YOUR_MANUAL_API_KEY"

✅ API Key successfully loaded and set as GOOGLE_API_KEY.


## 1.4 Document Upload

In [ ]:
print("\n--- Uploading Document: 'LenderFeesWorksheetNew.pdf' ---")
uploaded = files.upload()
pdf_path = None
if uploaded:
    pdf_path = list(uploaded.keys())[0]
    print(f"Successfully uploaded: {pdf_path}")
else:
    print("No file was uploaded. Exiting.")
    exit()



--- Uploading Document: 'LenderFeesWorksheetNew.pdf' ---


Saving LenderFeesWorksheetNew.pdf to LenderFeesWorksheetNew (2).pdf
Successfully uploaded: LenderFeesWorksheetNew (2).pdf


## 1.5 Custom PyMuPDF Loader Function (PDF Parsing)

In [ ]:
def load_pdf_with_pymupdf(pdf_path: str) -> List[Document]:
    """Load a PDF and convert it to LlamaIndex Document format using PyMuPDF."""
    doc = fitz.open(pdf_path)
    documents = []
    for i, page in enumerate(doc):
        text = page.get_text()
        if not text.strip(): continue # Skip empty pages
        documents.append(
            Document(text=text, metadata={"file_name": os.path.basename(pdf_path), "page_number": i + 1})
        )
    doc.close()
    print(f"Processed {pdf_path}: Extracted {len(documents)} pages with content.")
    return documents

## 1.6 Configure RAG Pipeline (LLM, Embeddings, Chunking)

In [ ]:
print("\n--- Configuring LlamaIndex Settings ---")
# LLM: Gemini 2.5 Flash
llm = Gemini(model="models/gemini-2.5-flash")
Settings.llm = llm

# Embedding Model: HuggingFace all-MiniLM-L6-v2 (Efficient and Local)
# Hugging Face API Key setup in Colab Secret
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = embed_model

# Chunking Strategy: SentenceSplitter with optimal settings
Settings.text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.chunk_size = 512
Settings.chunk_overlap = 20


--- Configuring LlamaIndex Settings ---


/tmp/ipython-input-3859728630.py:3: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(model="models/gemini-2.5-flash")


## 1.7 Indexing and Index Creation

In [ ]:
documents = load_pdf_with_pymupdf(pdf_path)
index = VectorStoreIndex.from_documents(documents)
print("✅ Indexing complete. RAG VectorStoreIndex created.")

Processed LenderFeesWorksheetNew (2).pdf: Extracted 1 pages with content.
✅ Indexing complete. RAG VectorStoreIndex created.


# **2. Reasons for Methods: Embedding, Chunking Strategy, & Retrieval Method**

## **🔢 Embedding Model**

**Model:** `sentence-transformers/all-MiniLM-L6-v2`
<br><br>

**Justification:** This model is an extremely efficient, small, and fast open-source embedding model. It provides a good balance of accuracy and speed, making it highly suitable for rapid RAG development in a Colab environment. Running a local HuggingFace model also helps to reduce API costs and latency compared to calling a remote embedding service.


---



## **✂️ Chunking Strategy**

**Strategy:** `SentenceSplitter` with `chunk_size = 512` and `chunk_overlap = 20`.
<br><br>

**Justification:** The `SentenceSplitter` breaks text primarily at logical sentence boundaries, which is ideal for a semi-structured document like a financial worksheet.

  * **Chunk Size (512 tokens):** Provides enough context for the LLM to perform calculations or detailed analysis from the retrieved text.
  
  * **Overlap (20 tokens):** A small overlap ensures that the context is maintained across the split points, preventing critical information from being cut off.


---



## **🔍 Retrieval Method**

**Method:** **Vector Retrieval** (Semantic Search)
<br><br>

**Justification:** Vector retrieval is used because it finds relevant document segments based on the semantic mening of the query. This is essential for documents where specific financial terms might be used (e.g., "lender's title insurance") that a user might query using a different phrase (e.g., "fee to protect the lender's interest").


---



# **3.Create Query Engine and Execute Prompts**

## Create Query Engine

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=2)

print("\n--- Step 4: Query Results ---")


--- Step 4: Query Results ---


## Prompt 1: "What is the total estimated monthly payment?"

In [ ]:
prompt_1 = "What is the total estimated monthly payment?"
print(f"Prompt 1: \"{prompt_1}\"")
response_1 = query_engine.query(prompt_1)
print(f"Response 1: {response_1.response}\n")

Prompt 1: "What is the total estimated monthly payment?"
Response 1: The total estimated monthly payment is $1,869.37.



The total estimated monthly payment is the sum of the Principal & Interest and any recurring monthly escrow items.


* Principal & Interest (P&I)
  * $1,869.58	was found in the "TOTAL ESTIMATED MONTHLY PAYMENT" section.

* Hazard Insurance (Monthly)
  * $39.58

  * The document lists a 12-month premium of $475.00

  * Calculated $39.58 x 12 mth(s)

* Total Estimated Monthly Payment
  * $1,909.16

  * Calculated by adding ($1,869.58 + $39.58)




## Prompt 2: "How much does the borrower pay for lender's title insurance?"

In [ ]:
# Prompt 2
prompt_2 = "How much does the borrower pay for lender's title insurance?"
print(f"Prompt 2: \"{prompt_2}\"")
response_2 = query_engine.query(prompt_2)
print(f"Response 2: {response_2.response}")

Prompt 2: "How much does the borrower pay for lender's title insurance?"
Response 2: The borrower pays $650.00 for Lender's Title Insurance.
